# Double pendulum using Lagrange's equation

Defines a DoublePendulum class that is used to generate basic pendulum plots from solving Lagrange's equations for a double pendulum.

* Last revised 25-Apr-2022 by Matt Shmukler (shmukler.2@osu.edu).

We take the origin of our system to be the top point where the pendulum is attached. $\hat x$ points to the right and $\hat y$ points down. Here is a picture of the setup and variables (borrowed from Taylor's Classical Mechanics).

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from IPython.display import Image
import matplotlib.pyplot as plt
Image(filename='DoublePendulumScreencap.png')

## Euler-Lagrange equations

For the double pendulum with generalized coordinates $\phi_1$ and $\phi_2$, we can write the position of the bobs as

$
x_1=L_1 \sin(\phi_1) \hspace{2cm} \dot x_1=L_1\dot\phi_1\cos(\phi_1) \\x_2=x_1+L_2 \sin(\phi_2) \hspace{1.05cm} \dot x_2=L_1\dot\phi_1\cos(\phi_1)+L_2\dot\phi_2\cos(\phi_2)\\y_1=L_1\cos(\phi_1) \hspace{2cm} \dot y_1=-L_1\dot\phi_1\sin(\phi_1) \\y_2=y_1+L_2\cos(\phi_2) \hspace{1.05cm} \dot y_2=-L_1\dot\phi_1\sin(\phi_1)-L_2\dot\phi_2\sin(\phi_2)
$

The kinetic and potenial energies are then

$
T=\frac{1}{2}m_1v_1^2+\frac{1}{2}m_2v_2^2=\frac{1}{2}m_1(\dot x_1^2+\dot y_1^2)+\frac{1}{2}m_2(\dot x_2^2+\dot y_2^2)=\frac{1}{2}(m_1+m_2)L_1^2\dot\phi_1^2+m_2L_1L_2\dot\phi_1\dot\phi_2\cos(\phi_1-\phi_2)+\frac{1}{2}m_2L_2^2\dot\phi_2^2
$
$
U=-m_1gy_1-m_2gy_2=-(m_1+m_2)L_1g\cos(\phi_1)-m_2L_2g\cos(\phi_2)\\
$
Which gives us a lagrangian of 
$\\
\mathcal{L}=\frac{1}{2}(m_1+m_2)L_1^2\phi_1^2+\frac{1}{2}m_2L_2^2\dot\phi_2^2+m_2L_1L_2\dot\phi_1\dot\phi_2\cos(\phi_1-\phi_2)+(m_1+m_2)L_1g\cos(\phi_1)+m_2L_2g\cos(\phi_2)\\
$
Solving the Euler-Lagrange Equations:
$
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot\phi_i} = \frac{\partial\mathcal L}{\partial\phi_i}
 \quad\Longrightarrow\quad
$
$\\
\ddot\phi_1=\frac{m_2g\sin(\phi_2)\cos(\phi_1-\phi_2)-m_2\sin(\phi_1-\phi_2)(L_1\dot\phi_1^2\cos(\phi_1-\phi_2)+L_2\dot\phi_2^2)-(m_1+m_2)g\sin(\phi_1)}{L_1(m_1+m_2\sin^2(\phi_1-\phi_2)))}\\
$
and
$\\
\ddot\phi_2=\frac{(m_1+m_2)(L_1\dot\phi_1^2\sin(\phi_1-\phi_2)-g\sin(\phi_2)+g\sin(\phi_1)\cos(\phi_1-\phi_2))+m_2L_2\dot\phi_2^2\sin(\phi_1-\phi_2)\cos(\phi_1-\phi_2)}{L_2(m_1+m_2\sin^2(\phi_1-\phi_2)))}
\\
$
We shall now solve these equations numerically

In [ ]:
# The dpi (dots-per-inch) setting will affect the resolution and how large
#  the plots appear on screen and printed.  So you may want/need to adjust 
#  the figsize when creating the figure.
plt.rcParams['figure.dpi'] = 100.    # this is the default for notebook

# Change the common font size (smaller when higher dpi)
font_size = 12
plt.rcParams.update({'font.size': font_size})

## Double Pendulum class and utility functions

In [ ]:
class DoublePendulum():
    """
    DoublePendulum class implements the parameters and Lagranges's equations for 
     a double pendulum (no driving or damping).
     
    Parameters
    ----------
    Ln : float
        length of the n'th pendulum
    g : float
        gravitational acceleration at the earth's surface
    mn : float
        mass of the n'th pendulum

    Methods
    -------
    dy_dt(t, y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self, L1=1., L2=1., m1=1., m2=1., g=1.):
        #Initialize members of class
        self.L1 = L1
        self.L2 = L2
        self.g = g
        self.m1 = m1
        self.m2 = m2
    
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi1/dt dphi_dot1/dt dphi2/dt dphi_dot2/dt]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            A 2-component vector with y[0] = phi1(t) and y[1] = phi_dot1(t)
            and y[2] = phi2(t) and y[3] = phi_dot2(t)
            
        Returns
        -------
        
        """
        cosp , sinp = np.cos(y[0]-y[2]), np.sin(y[0]-y[2])
        d = (self.m1+self.m2 * sinp**2)
        
        phi1_doubledot= (self.m2*self.g*np.sin(y[2])*cosp-self.m2*sinp*(self.L1 * y[1]**2 * cosp +self.L2 * y[3]**2) \
                         - (self.m1+self.m2)*self.g*np.sin(y[0]))/(self.L1*d)
        
        phi2_doubledot= ((self.m1+self.m2)*(self.L1 * y[1]**2 * sinp - self.g*np.sin(y[2])+self.g*np.sin(y[0])*cosp) \
                          +self.m2*self.L2* y[3]**2 * sinp*cosp)/(self.L2*d)
        
        
        return y[1], phi1_doubledot  , y[3], phi2_doubledot 
    
    def solve_ode(self, t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0,
                  abserr=1.0e-10, relerr=1.0e-10):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [phi1_0, phi1_dot_0, phi2_0, phi2_dot_0] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        phi1, phi1_dot, phi2, phi2_dot = solution.y

        return phi1, phi1_dot, phi2, phi2_dot

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [ ]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

## Make pendulum plots of the two angles

In [ ]:
# Labels for individual plot axes
phi_vs_time_labels = (r'$t$', r'$\phi(t)$')

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 50.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

L1 = 1.
L2 = 1.
g = 1.
m1 = 1.
m2 = 1.

# Instantiate a pendulum 
p1 = DoublePendulum(L1=L1, L2=L2, m1=m1, m2=m2 ,g=g)


In [ ]:
# initial conditions
phi1_0 = np.pi/2.
phi1_dot_0 = 0.
phi2_0 = np.pi
phi2_dot_0 = 0.
phi1, phi1_dot, phi2, phi2_dot = p1.solve_ode(t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0)


# start the plot!
fig = plt.figure(figsize=(10,5))
overall_title = 'Double pendulum from Lagrangian:  ' + \
                rf' $\phi_1(0) = {phi1_0:.2f},$' + \
                rf'  $\dot\phi_1(0) = {phi1_dot_0:.2f},$' + \
                rf' $\phi_2(0) = {phi2_0:.2f}$' + \
                rf' $\dot\phi_2(0) = {phi2_dot_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
# first plot: phi_1(t) and phi_2(t) plot
ax_a = fig.add_subplot(1,1,1)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], phi1[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=r'$\phi_1(t)$', 
            title=None, 
            ax=ax_a) 
plot_y_vs_x(t_pts[start : stop], phi2[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='red',
            label=r'$\phi_2(t)$', 
            title=None, 
            ax=ax_a)  
                                
fig.tight_layout()
fig.savefig('Double_pendulum_Lagrange.png', bbox_inches='tight')  

## Chaos of the double pendulum

In [ ]:
# initial conditions
phi1_0 = np.pi/2.
phi1_dot_0 = 0.
phi2_0 = np.pi
phi2_dot_0 = 0.
phi1, phi1_dot, phi2, phi2_dot = p1.solve_ode(t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0)
#Start second pendulum with slightly different initial conditions
phi1s, phi1_dots, phi2s, phi2_dots = p1.solve_ode(t_pts, phi1_0, phi1_dot_0, phi2_0-0.001, phi2_dot_0)

# start the plot!
fig2 = plt.figure(figsize=(10,5))
overall_title = 'Double pendulum from Lagrangian:  ' + \
                rf' $\phi_1(0) = {phi1_0:.2f},$' + \
                rf'  $\dot\phi_1(0) = {phi1_dot_0:.2f},$' + \
                rf' $\phi_2(0) = {phi2_0:.2f}$' + \
                rf' $\dot\phi_2(0) = {phi2_dot_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
fig2.suptitle(overall_title, va='baseline')

ax_a = fig2.add_subplot(1,1,1)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], np.abs(phi1[start : stop]-phi1s[start : stop]), 
            axis_labels=phi_vs_time_labels, 
            color='blue', semilogy=True,
            label=r'$\Delta\phi_1(t)$', 
            title=None, 
            ax=ax_a) 
plot_y_vs_x(t_pts[start : stop], np.abs(phi2[start : stop]-phi2s[start : stop]), 
            axis_labels=phi_vs_time_labels, 
            color='red', semilogy=True,
            label=r'$\Delta\phi_2(t)$', 
            title=None, 
            ax=ax_a)  
                                
fig2.tight_layout()
fig2.savefig('Double_pendulum_Chaos.png', bbox_inches='tight')  

For the plot above, we solve for the motion of the pendulum with two slightly different initial conditions  ($\phi_2(0)$ varies by 0.001) and then plot the absolute value of the difference between the two different $\phi_1$'s and $\phi_2$'s. We can see that for both $\phi_1$ and $\phi_2$, the difference between the two motions grows in time until the motion is completely different. This growth is indicative of a positive lyapunov exponent and means that slight differences in initial conditions lead to significant differences in the motions of our pendulums over time. This sensitivity to initial conditions shows that the system is chaotic.